#### 从数据库导入数据

In [1]:
import pymysql.cursors
import pandas as pd

db = pymysql.connect(host='localhost',user='root',password='root',db='lagou',charset='utf8',cursorclass=pymysql.cursors.DictCursor)
cursor = db.cursor()
sql = 'SELECT * FROM dataset;'
cursor.execute(sql)
result = cursor.fetchall()
df = pd.DataFrame(list(result))
db.close()

df

,jobNature,companySize,workYear,education,industryField,thirdType,salaryMin,salaryMax,nameUnfit,positionId,...,financeStage,firstType,secondType,district,salary,resumeProcessRate,resumeProcessDay,score,famousCompany,isFocus
0,全职,500-2000人,1-3年,本科,旅游,数据分析,18,35,0,6112946,...,D轮及以上,开发|测试|运维类,数据开发,福田区,18k-35k,100,1,199,0,1
1,全职,500-2000人,3-5年,本科,"消费生活,移动互联网",商业,15,25,0,6229664,...,C轮,产品|需求|项目类,数据分析,南山区,15k-25k,100,1,179,0,1
2,全职,2000人以上,5-10年,本科,金融,建模,15,30,0,6853427,...,上市公司,开发|测试|运维类,数据开发,福田区,15k-30k,100,1,130,1,0
3,全职,2000人以上,1-3年,本科,金融,Hadoop,20,30,0,6692363,...,不需要融资,开发|测试|运维类,数据开发,南山区,20k-30k,100,2,89,0,0
4,全职,2000人以上,1-3年,大专,"移动互联网,游戏",数据分析,8,10,0,6835783,...,不需要融资,产品|需求|项目类,数据分析,南山区,8k-10k,0,1,69,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,全职,2000人以上,3-5年,本科,社交,产品经理,15,25,1,5276304,...,上市公司,产品|需求|项目类,产品经理,南山区,15k-25k,100,1,10,1,0
271,全职,2000人以上,3-5年,本科,社交,数据挖掘,18,35,0,6842800,...,上市公司,开发|测试|运维类,数据开发,南山区,18k-35k,100,1,10,1,1
272,全职,2000人以上,3-5年,本科,社交,数据分析,20,40,0,5250466,...,上市公司,产品|需求|项目类,数据分析,南山区,20k-40k,100,1,10,1,1
273,全职,2000人以上,1-3年,本科,金融,数据分析,20,30,0,6391267,...,未融资,开发|测试|运维类,数据开发,南山区,20k-30k,15,1,4,1,0


#### 拆分训练集和测试集

In [2]:
from sklearn.model_selection import train_test_split
X = df.iloc[:,:-1]
X = pd.get_dummies(X).values
y = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [3]:
X_names = pd.get_dummies(df.iloc[:,:-1]).columns
X_names

Index(['salaryMin', 'salaryMax', 'nameUnfit', 'resumeProcessRate',
       'resumeProcessDay', 'score', 'famousCompany', 'jobNature_全职',
       'jobNature_兼职', 'jobNature_实习',
       ...
       'salary_7k-13k', 'salary_7k-14k', 'salary_8k-10k', 'salary_8k-11k',
       'salary_8k-12k', 'salary_8k-13k', 'salary_8k-15k', 'salary_8k-16k',
       'salary_9k-12k', 'salary_9k-14k'],
      dtype='object', length=687)

#### 标准化处理

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[-0.73480553, -0.61025744, -0.53545615, ..., -0.07392213,
        -0.10482848, -0.07392213],
       [ 1.62260134,  1.664301  , -0.53545615, ..., -0.07392213,
        -0.10482848, -0.07392213],
       [ 0.23589142,  0.14792871, -0.53545615, ..., -0.07392213,
        -0.10482848, -0.07392213],
       ...,
       [ 0.23589142,  0.52702178, -0.53545615, ..., -0.07392213,
        -0.10482848, -0.07392213],
       [ 0.23589142,  0.14792871,  1.86756657, ..., -0.07392213,
        -0.10482848, -0.07392213],
       [-0.45746355, -0.23116436, -0.53545615, ..., -0.07392213,
        -0.10482848, -0.07392213]])

#### 网格搜索（含交叉验证）——找出最优参数

In [8]:
?GridSearchCV

In [9]:
?MLPClassifier

In [16]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

clf = MLPClassifier()
parameters = {
    'random_state':[0],
    'activation':['identity', 'logistic', 'tanh', 'relu'],
    'solver':['lbfgs', 'sgd', 'adam'],
    'learning_rate':['constant', 'invscaling', 'adaptive'],
    'max_iter':[5000]
}
scorings = ['roc_auc', 'recall']
for scoring in scorings:
    print('===========================================================================')
    print('此次调参的scoring为：%s' % scoring)
    print()
    clf = GridSearchCV(clf, parameters, scoring=scoring, cv=10)
    clf.fit(X_train, y_train)
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    params = clf.cv_results_['params']
    print('【交叉验证】过程如下：')
    for mean, std, param in zip(means, stds, params):
        print('%s得分为 %0.3f(+-%0.3f)时，参数为：%r' % (scoring, mean, std * 2, param))
    print()
    print('【训练集】最佳参数为：%r' % clf.best_params_)
    print('【训练集】最佳参数得分为：%0.3f' % clf.best_score_)
    print()
    print('【测试集】%s得分为：%0.3f' % (scoring, clf.score(X_test, y_test)))    # 直接获取测试集的得分
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    reports = classification_report(y_true, y_pred)
    print('【测试集中实际值与预测值对比】模型报告如下：')
    print(reports)

此次调参的scoring为：roc_auc

【交叉验证】过程如下：
roc_auc得分为 0.865(+-0.144)时，参数为：{'activation': 'identity', 'learning_rate': 'constant', 'max_iter': 5000, 'random_state': 0, 'solver': 'lbfgs'}
roc_auc得分为 0.823(+-0.210)时，参数为：{'activation': 'identity', 'learning_rate': 'constant', 'max_iter': 5000, 'random_state': 0, 'solver': 'sgd'}
roc_auc得分为 0.654(+-0.213)时，参数为：{'activation': 'identity', 'learning_rate': 'constant', 'max_iter': 5000, 'random_state': 0, 'solver': 'adam'}
roc_auc得分为 0.865(+-0.144)时，参数为：{'activation': 'identity', 'learning_rate': 'invscaling', 'max_iter': 5000, 'random_state': 0, 'solver': 'lbfgs'}
roc_auc得分为 0.523(+-0.307)时，参数为：{'activation': 'identity', 'learning_rate': 'invscaling', 'max_iter': 5000, 'random_state': 0, 'solver': 'sgd'}
roc_auc得分为 0.654(+-0.213)时，参数为：{'activation': 'identity', 'learning_rate': 'invscaling', 'max_iter': 5000, 'random_state': 0, 'solver': 'adam'}
roc_auc得分为 0.865(+-0.144)时，参数为：{'activation': 'identity', 'learning_rate': 'adaptive', 'max_iter': 5000, 'r

C:\Users\romak\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


ValueError: Invalid parameter activation for estimator GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_iter=200,
                                     momentum=0.9, n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_st...
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'activation': ['identity', 'logistic', 'tanh', 'relu'],
             

奇怪，scorings每次只能用一个参数的？